<a href="https://colab.research.google.com/github/danchaud-vincent/tensorflow-deep-learning/blob/main/04_Transfer_learning_in_tensorflow_part1_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04 - Transer learning in tensorflow part 1 Feature Extraction